## Fine tuning Bert model

In [16]:
%pip install --upgrade huggingface_hub
%pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoTokenizer, AutoModelForPreTraining
import torch


In [1]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("Maltehb/danish-bert-botxo")
model = AutoModelForPreTraining.from_pretrained("Maltehb/danish-bert-botxo")


In [2]:
print(tokenizer("hej jeg hedder lucas"))

{'input_ids': [2, 2175, 82, 2567, 21100, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [2]:
import stanza
stanza.download("da")

2022-12-15 12:28:29 INFO: Downloading default packages for language: da (Danish) ...
2022-12-15 12:28:30 INFO: File exists: /Users/lucasvilsen/stanza_resources/da/default.zip
2022-12-15 12:28:32 INFO: Finished downloading models and saved to /Users/lucasvilsen/stanza_resources.


In [1]:
import stanza
pos_model = stanza.Pipeline("da")
text = "i kan godt lide at spiser mad fra carlsberg i københavn, danmark"
result = pos_model(text)
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in result.sentences for word in sent.words], sep='\n')

2022-12-16 05:41:36 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-16 05:41:38 INFO: Loading these models for language: da (Danish):
| Processor    | Package   |
----------------------------
| tokenize     | ddt       |
| pos          | ddt       |
| lemma        | ddt       |
| depparse     | ddt       |
| constituency | arboretum |
| ner          | ddt       |

2022-12-16 05:41:38 INFO: Use device: cpu
2022-12-16 05:41:38 INFO: Loading: tokenize
2022-12-16 05:41:38 INFO: Loading: pos
2022-12-16 05:41:38 INFO: Loading: lemma
2022-12-16 05:41:38 INFO: Loading: depparse
2022-12-16 05:41:38 INFO: Loading: constituency
2022-12-16 05:41:39 INFO: Loading: ner
2022-12-16 05:41:39 INFO: Done loading processors!


word: i	upos: ADP	xpos: None	feats: AdpType=Prep
word: kan	upos: AUX	xpos: None	feats: Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act
word: godt	upos: ADV	xpos: None	feats: Degree=Pos
word: lide	upos: VERB	xpos: None	feats: VerbForm=Inf|Voice=Act
word: at	upos: PART	xpos: None	feats: PartType=Inf
word: spiser	upos: VERB	xpos: None	feats: Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act
word: mad	upos: NOUN	xpos: None	feats: Definite=Ind|Gender=Com|Number=Sing
word: fra	upos: ADP	xpos: None	feats: AdpType=Prep
word: carlsberg	upos: PROPN	xpos: None	feats: _
word: i	upos: ADP	xpos: None	feats: AdpType=Prep
word: københavn	upos: PROPN	xpos: None	feats: _
word: ,	upos: PUNCT	xpos: None	feats: _
word: danmark	upos: PROPN	xpos: None	feats: _


In [3]:
dictionary = {}
for sent in result.sentences:
    for word in sent.words:
        dictionary[word.text] = word.upos
print(dictionary)

{'i': 'ADP', 'kan': 'AUX', 'godt': 'ADV', 'lide': 'VERB', 'at': 'PART', 'spiser': 'VERB', 'mad': 'NOUN', 'fra': 'ADP', 'carlsberg': 'PROPN', 'københavn': 'PROPN', ',': 'PUNCT', 'danmark': 'PROPN'}


In [65]:

my_lst = [
['hej', 'Hej', 0, '"hej" skal begynde med stort bogstav: "Hej", da "hej" er det første ord i en ny sætning.'],
['per', 'Per', 3, '"per" skal begynde med stort bogstav: "Per", da "Per" er et egenavn.'],
['hedder', 'hedder,', 2, 'Der skal være komma efter "hedder"'],
['Per', 'Per.', 3, 'Der skal være punktum efter "Per".'],
['jeg', 'Jeg', 4, '"jeg" skal begynde med stort bogstav: "Jeg", da "jeg" er det første ord i en ny sætning.'],
['carlsberg', 'Carlsberg', 12, '"carlsberg" skal begynde med stort bogstav: "Carlsberg", da "Carlsberg" er et egenavn.'],
['københavn', 'København', 14, '"københavn" skal begynde med stort bogstav: "København", da "København" er et egenavn.'],
['København', 'København.', 14, 'Der skal være punktum efter "København".']]

In [66]:
def concat_duplicates(lst):
    elements = {}
    unique_lst = []
    for sublist in lst:
        if sublist[2] in elements.keys():
            elements[sublist[2]][1] = sublist[1]
            elements[sublist[2]][3] += " " + sublist[3]
        else:
            elements[sublist[2]] = sublist
    return elements.values()

print(*concat_duplicates(my_lst), sep="\n")

['hej', 'Hej', 0, '"hej" skal begynde med stort bogstav: "Hej", da "hej" er det første ord i en ny sætning.']
['per', 'Per.', 3, '"per" skal begynde med stort bogstav: "Per", da "Per" er et egenavn. Der skal være punktum efter "Per".']
['hedder', 'hedder,', 2, 'Der skal være komma efter "hedder"']
['jeg', 'Jeg', 4, '"jeg" skal begynde med stort bogstav: "Jeg", da "jeg" er det første ord i en ny sætning.']
['carlsberg', 'Carlsberg', 12, '"carlsberg" skal begynde med stort bogstav: "Carlsberg", da "Carlsberg" er et egenavn.']
['københavn', 'København.', 14, '"københavn" skal begynde med stort bogstav: "København", da "København" er et egenavn. Der skal være punktum efter "København".']


In [62]:
def concat_duplicates(lst):
    to_pop = []
    for i in range(len(lst)):
        for j in range(len(lst)):
            if (i == j) or (i > j):
                continue
            if lst[i][2] == lst[j][2]:
                wrong_word = lst[i][0]
                right_word =lst[j][1]
                index = lst[i][2]
                description = lst[i][3] + " " + lst[j][3]
                lst[i] = [wrong_word, right_word, index, description]
                to_pop.append(j)
    for value in to_pop:
        lst.pop(value)
    return lst

new_list = concat_duplicates(my_lst)
print(*new_list, sep="\n")

IndexError: pop index out of range

True